<a href="https://colab.research.google.com/github/Bluseass/Imdb-semantic-analyzer/blob/main/imdb_semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb

In [ ]:
# Загрузка данных IMDb
num_words = 20000
maxlen = 200

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
# Паддинг последовательностей
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

# Создание объекта токенизатора
tokenizer = keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_sequences(x_train)

In [ ]:
# Создание модели Transformer
embedding_dim = 128
num_heads = 8
dff = 512
num_layers = 6

inputs = layers.Input(shape=(maxlen,))
embedding_layer = layers.Embedding(num_words, embedding_dim)(inputs)
position_embeddings = layers.Embedding(input_dim=maxlen, output_dim=embedding_dim)(tf.range(start=0, limit=maxlen, delta=1))
embeddings = embedding_layer + position_embeddings

# Добавление dropout
dropout_layer = layers.Dropout(0.1)(embeddings)

# Добавление нескольких блоков трансформера
for _ in range(num_layers):
    # Мульти-головное внимание
    attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim // num_heads)(dropout_layer, dropout_layer, dropout_layer)
    attention_output = layers.Dropout(0.1)(attention_output)
    attention_output = layers.LayerNormalization(epsilon=1e-6)(dropout_layer + attention_output)

    # Полносвязный слой
    ffn_output = layers.Dense(dff, activation='relu')(attention_output)
    ffn_output = layers.Dense(embedding_dim)(ffn_output)
    ffn_output = layers.Dropout(0.1)(ffn_output)
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(attention_output + ffn_output)
    dropout_layer = ffn_output

In [ ]:
# глобальный пулинг
pooling_output = layers.GlobalMaxPooling1D()(dropout_layer)
outputs = layers.Dense(1, activation='sigmoid')(pooling_output)

# сборка и обучение
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=64, epochs=5)

In [ ]:
from ipywidgets import widgets

def analyze_text(text):
    # Предварительная обработка текста
    text_sequence = tokenizer.texts_to_sequences([text])
    text_padded = keras.preprocessing.sequence.pad_sequences(text_sequence, maxlen=maxlen)

    # Получение прогноза от модели Transformer
    prediction = model.predict(text_padded)
    sentiment = "Положительный" if prediction > 0.5 else "Отрицательный"

    return f"Результат анализа: {sentiment}"


def analyze_button_clicked(b):
    result = analyze_text(input_text.value)
    output_text.value = result


def clear_button_clicked(b):
    input_text.value = ''
    output_text.value = ''

# Создание текстового поля для ввода текста
input_text = widgets.Textarea(description='Текст:', layout={'width': '400px', 'height': '200px'})

# Создание кнопки для запуска анализа текста
analyze_button = widgets.Button(description='Анализировать')
analyze_button.on_click(analyze_button_clicked)

# Создание кнопки для очистки поля
clear_button = widgets.Button(description='Очистить')
clear_button.on_click(clear_button_clicked)

# Создание текстового поля для вывода результата анализа
output_text = widgets.Textarea(description='Результат:', layout={'width': '400px', 'height': '50px'})

# Отображение виджетов
display(input_text)
display(widgets.HBox([analyze_button, clear_button]))
display(output_text)